In [31]:
#import all necessary libraries
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [32]:
df=pd.read_csv('EV_dataset.csv')
df.head()

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,September 30 2022,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,December 31 2022,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,January 31 2020,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,June 30 2022,Ferry,WA,Truck,0,0,0,"3,575","3,575",0.00
4,July 31 2021,Douglas,CO,Passenger,0,1,1,83,84,1.19


In [33]:
df.shape

(20819, 10)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20819 entries, 0 to 20818
Data columns (total 10 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Date                                      20819 non-null  object 
 1   County                                    20733 non-null  object 
 2   State                                     20733 non-null  object 
 3   Vehicle Primary Use                       20819 non-null  object 
 4   Battery Electric Vehicles (BEVs)          20819 non-null  object 
 5   Plug-In Hybrid Electric Vehicles (PHEVs)  20819 non-null  object 
 6   Electric Vehicle (EV) Total               20819 non-null  object 
 7   Non-Electric Vehicle Total                20819 non-null  object 
 8   Total Vehicles                            20819 non-null  object 
 9   Percent Electric Vehicles                 20819 non-null  float64
dtypes: float64(1), object(9)
memory us

In [35]:
df.isnull().sum()

Date                                         0
County                                      86
State                                       86
Vehicle Primary Use                          0
Battery Electric Vehicles (BEVs)             0
Plug-In Hybrid Electric Vehicles (PHEVs)     0
Electric Vehicle (EV) Total                  0
Non-Electric Vehicle Total                   0
Total Vehicles                               0
Percent Electric Vehicles                    0
dtype: int64

In [36]:
# Compute Q1, Q2, Q3 AND Q4
Q1 = df['Percent Electric Vehicles'].quantile(0.25)
Q2 = df['Percent Electric Vehicles'].quantile(0.50)
Q3 = df['Percent Electric Vehicles'].quantile(0.75)
Q4 = df['Percent Electric Vehicles'].quantile(1.00)
print('Q1:', Q1)
print('Q2:', Q2)
print('Q3:', Q3)
print('Q4:', Q4)
IQR = Q3 - Q1

# Define outlier boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
print('lower_bound:', lower_bound)
print('upper_bound:', upper_bound)

# Identify outliers
outliers = df[(df['Percent Electric Vehicles'] < lower_bound) | (df['Percent Electric Vehicles'] > upper_bound)]
print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])

Q1: 0.39
Q2: 1.22
Q3: 2.995
Q4: 100.0
lower_bound: -3.5174999999999996
upper_bound: 6.9025
Number of outliers in 'Percent Electric Vehicles': 2476


In [37]:
# Convert 'Date' to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Remove rows where 'Date' conversion failed
df = df[df['Date'].notnull()]

# Clean and convert numeric columns
numeric_columns = [
    "Battery Electric Vehicles (BEVs)",
    "Plug-In Hybrid Electric Vehicles (PHEVs)",
    "Electric Vehicle (EV) Total",
    "Non-Electric Vehicle Total",
    "Total Vehicles"
]

for col in numeric_columns:
    df[col] = df[col].astype(str).str.replace(',', '', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check the updated datatypes
print("\nData types after cleaning numeric columns:\n")
print(df[numeric_columns].dtypes)



Data types after cleaning numeric columns:

Battery Electric Vehicles (BEVs)            int64
Plug-In Hybrid Electric Vehicles (PHEVs)    int64
Electric Vehicle (EV) Total                 int64
Non-Electric Vehicle Total                  int64
Total Vehicles                              int64
dtype: object


In [38]:
# Fill missing 'County' and 'State' with mode (most frequent value)
df['County'].fillna(df['County'].mode()[0], inplace=True)
df['State'].fillna(df['State'].mode()[0], inplace=True)

# Check for remaining missing values
print("\nMissing values after filling 'County' and 'State':\n")
print(df[['County', 'State']].isnull().sum())



Missing values after filling 'County' and 'State':

County    0
State     0
dtype: int64


C:\Users\sarah\AppData\Local\Temp\ipykernel_23420\2393736436.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['County'].fillna(df['County'].mode()[0], inplace=True)
C:\Users\sarah\AppData\Local\Temp\ipykernel_23420\2393736436.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

In [39]:
# Calculate quantile caps
lower = df['Percent Electric Vehicles'].quantile(0.01)
upper = df['Percent Electric Vehicles'].quantile(0.99)

# Cap the outliers (Winsorization)
df['Percent Electric Vehicles'] = df['Percent Electric Vehicles'].clip(lower, upper)

# Print capped value range and number of remaining outliers
print("\nCapped 'Percent Electric Vehicles' to 1st–99th percentile:")
print(f"New min: {df['Percent Electric Vehicles'].min():.2f}, max: {df['Percent Electric Vehicles'].max():.2f}")

# Check outliers beyond new bounds
outliers = df[(df['Percent Electric Vehicles'] < lower) | (df['Percent Electric Vehicles'] > upper)]
print(f"Remaining outliers outside quantiles: {outliers.shape[0]}")



Capped 'Percent Electric Vehicles' to 1st–99th percentile:
New min: 0.00, max: 50.00
Remaining outliers outside quantiles: 0


In [41]:
# Summary of cleaned dataset
print("\nFinal dataset info after preprocessing:\n")
print(df.info())

# Preview a few rows
print("\nPreview of cleaned data:\n")
print(df.head())



Final dataset info after preprocessing:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20819 entries, 0 to 20818
Data columns (total 10 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   Date                                      20819 non-null  datetime64[ns]
 1   County                                    20819 non-null  object        
 2   State                                     20819 non-null  object        
 3   Vehicle Primary Use                       20819 non-null  object        
 4   Battery Electric Vehicles (BEVs)          20819 non-null  int64         
 5   Plug-In Hybrid Electric Vehicles (PHEVs)  20819 non-null  int64         
 6   Electric Vehicle (EV) Total               20819 non-null  int64         
 7   Non-Electric Vehicle Total                20819 non-null  int64         
 8   Total Vehicles                            20819 non-null  int64 